In [1]:
import pandas as pd
import numpy as np

In [10]:
cbr1krr = pd.read_csv('cbr1kr.csv')
hayabusa = pd.read_csv('hayabusa.csv')

cbr = cbr1krr.to_numpy()
haya = hayabusa.to_numpy()

cbr_radsec_Nm = cbr[:,[1,4]]

In [ ]:
# constants
mass = 500 #kg
wheel_rad = .3 #m
rear_weight = .55 #fraction out of 1
tire_mu = 1
time_step = .05 #sec


In [ ]:
#accel calc
time = np.linspace(0,15,1500)
vel = np.zeros_like(time)

